<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-28 11:19:03
-------------------
qualified stocks: 89
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.00 K
Current:  1.45 C
-------------------
Today PnL: -45.28 K (-0.31%)
Current PnL: -19.47 L (-12.76%)
CY Booked + Current PnL: -7.37 L (-4.83%)
-------------------
Total profit:  2.36 L
Total loss:  -21.83 L
-------------------
Total Booked + Current PnL: 19.61 L (15.61%)
Total Booked PnL: 39.08 L (31.11%)
Curr Year Booked PnL: 12.10 L (8.36%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.37 L (60.38%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-7.66,72.0,X-LC,3.82,231317.0,20951.0,7842.0,-0.09,9.96,3.39,13.68,37.0,2.67,1.60,28.13,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,99.79,58.0,M-SC,1.96,86087.0,-14690.0,14781.0,1.58,-14.58,17.17,0.09,245.0,-0.99,0.60,13.10,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.0,-9.24,44.0,X-MC,2.98,89760.0,-918.0,15825.0,0.50,-1.01,17.63,16.44,101.0,-0.06,0.62,18.96,X40,ATH,PHARMA
42,ITC,452.0,-36.66,62.0,X-LC,1.55,204206.0,4068.0,16827.0,-0.73,2.03,8.24,10.44,4.0,0.24,1.41,7.71,X40,NTT,FMCG
86,VOLTAS,1530.0,1.02,52.0,X-MC,3.53,211860.0,20118.0,17648.0,-1.68,10.49,8.33,19.69,99.0,1.14,1.46,18.00,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.0,-44.35,54.0,X-SC,38.31,55232.0,-9774.0,160703.0,3.68,-15.04,290.96,232.18,198.0,-0.06,0.38,7.73,XY24,NTT,MISC
56,RAJOOENG,143.1,-41.82,47.0,H-SC,18.36,88080.0,-14420.0,55024.0,3.25,-14.07,62.47,39.61,114.0,-0.26,0.61,7.57,AR,ATH,MISC
84,VALIANTORG,838.0,-290.89,45.0,H-SC,8.69,106046.0,-57559.0,178878.0,2.36,-35.18,168.68,74.15,139.0,-0.32,0.73,35.93,XR,NTT,CHEMICALS
75,TATAMOTORS,1065.0,-53.17,19.0,X-LC,16.99,148848.0,-127675.0,237740.0,1.67,-46.17,159.72,39.81,54.0,-0.54,1.03,4.91,XY24,NTT,AUTO
62,SAIL,228.0,44.35,53.0,M-MC,9.75,227356.0,2394.0,165265.0,1.64,1.06,72.69,74.53,192.0,0.01,1.57,33.66,XY24,BTT,STEEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-37.29,38.0,X-SC,4.44,93790.0,-34880.0,80181.0,-3.10,-27.11,85.49,35.20,219.0,-0.44,0.65,7.65,X40,NTT,FOOTWEAR
15,BSOFT,836.99,-14.41,59.0,H-SC,13.96,96288.0,-44365.0,117144.0,-2.35,-31.54,121.66,51.74,131.0,-0.38,0.67,11.40,XR,ATH,IT
32,HINDZINC,730.22,25.96,45.0,M-LC,11.18,205146.0,70.0,112502.0,-2.22,0.03,54.84,54.89,52.0,0.00,1.42,24.14,X5K,ATH,METALS
8,ATULAUTO,844.00,3564.86,46.0,M-SC,3.43,164560.0,-32572.0,122400.0,-1.81,-16.52,74.38,45.57,236.0,-0.27,1.14,16.67,XY24,NTT,AUTO
5,ANGELONE,3033.00,17.35,67.0,X-SC,5.33,207640.0,16634.0,41071.0,-1.74,8.71,19.78,30.21,157.0,0.41,1.44,31.41,X40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.00,127.54,69.0,H-SC,17.42,143883.0,1404.0,18115.0,-1.31,0.99,12.59,13.70,163.0,0.08,0.99,69.82,OX40N,NTT,BANKS
36,IEX,219.00,-30.39,61.0,H-SC,16.45,198733.0,-1221.0,98452.0,-0.47,-0.61,49.54,48.63,115.0,-0.01,1.37,10.87,XR,NTT,MISC
57,RECLTD,446.00,45.36,50.0,M-LC,5.49,203775.0,605.0,41529.0,-0.86,0.30,20.38,20.74,55.0,0.01,1.41,7.18,XY25,NTT,FINANCE
88,WIPRO,420.00,-14.70,45.0,M-LC,5.78,150263.0,-682.0,110143.0,-0.64,-0.45,73.30,72.51,53.0,-0.01,1.04,5.54,XR,NTT,IT
32,HINDZINC,730.22,25.96,45.0,M-LC,11.18,205146.0,70.0,112502.0,-2.22,0.03,54.84,54.89,52.0,0.00,1.42,24.14,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.0,127.54,69.0,H-SC,17.42,143883.0,1404.0,18115.0,-1.31,0.99,12.59,13.70,163.0,0.08,0.99,69.82,OX40N,NTT,BANKS
79,TTKPRESTIG,770.0,99.79,58.0,M-SC,1.96,86087.0,-14690.0,14781.0,1.58,-14.58,17.17,0.09,245.0,-0.99,0.60,13.10,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.42,50.0,H-SC,2.60,224289.0,-45378.0,81708.0,-1.57,-16.83,36.43,13.47,138.0,-0.56,1.55,14.04,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.37,54.0,H-MC,7.05,104994.0,-26841.0,69065.0,-1.07,-20.36,65.78,32.03,98.0,-0.39,0.73,17.60,OX40N,NTT,IT
18,CERA,9475.0,-24.11,41.0,H-SC,2.62,138184.0,-37719.0,79746.0,0.18,-21.44,57.71,23.89,149.0,-0.47,0.96,19.34,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.00,127.54,69.0,H-SC,17.42,143883.0,1404.0,18115.0,-1.31,0.99,12.59,13.70,163.0,0.08,0.99,69.82,OX40N,NTT,BANKS
57,RECLTD,446.00,45.36,50.0,M-LC,5.49,203775.0,605.0,41529.0,-0.86,0.30,20.38,20.74,55.0,0.01,1.41,7.18,XY25,NTT,FINANCE
62,SAIL,228.00,44.35,53.0,M-MC,9.75,227356.0,2394.0,165265.0,1.64,1.06,72.69,74.53,192.0,0.01,1.57,33.66,XY24,BTT,STEEL
32,HINDZINC,730.22,25.96,45.0,M-LC,11.18,205146.0,70.0,112502.0,-2.22,0.03,54.84,54.89,52.0,0.00,1.42,24.14,X5K,ATH,METALS
47,KPIGREEN,731.05,8.64,67.0,H-SC,5.27,129629.0,4332.0,54600.0,-1.49,3.46,42.12,47.03,141.0,0.08,0.90,62.31,MH,ATH,POWER


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-16.77,74.0,H-MC,15.18,214889.0,35081.0,77339.0,0.01,19.51,35.99,62.52,88.0,0.45,1.49,60.25,XR,NTT,BANKS
87,WHIRLPOOL,2270.00,-38.39,64.0,M-SC,3.17,127779.0,12981.0,81063.0,-0.17,11.31,63.44,81.92,223.0,0.16,0.88,51.27,XR,NTT,DURABLES
80,UJJIVANSFB,60.00,127.54,69.0,H-SC,17.42,143883.0,1404.0,18115.0,-1.31,0.99,12.59,13.70,163.0,0.08,0.99,69.82,OX40N,NTT,BANKS
47,KPIGREEN,731.05,8.64,67.0,H-SC,5.27,129629.0,4332.0,54600.0,-1.49,3.46,42.12,47.03,141.0,0.08,0.90,62.31,MH,ATH,POWER
37,INDIAMART,4810.62,-53.52,56.0,H-SC,6.88,129988.0,6652.0,124970.0,-0.10,5.39,96.14,106.72,119.0,0.05,0.90,30.03,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-53.17,19.0,X-LC,16.99,148848.0,-127675.0,237740.0,1.67,-46.17,159.72,39.81,54.0,-0.54,1.03,4.91,XY24,NTT,AUTO
24,DMART,5391.45,-17.07,36.0,X-LC,10.00,101405.0,-1181.0,27988.0,-0.77,-1.15,27.60,26.13,38.0,-0.04,0.70,24.13,X40N,ATH,FMCG
45,JSWINFRA,342.00,-20.69,36.0,X-MC,5.38,199091.0,-1256.0,31755.0,-0.35,-0.63,15.95,15.23,178.0,-0.04,1.38,31.42,X40N,NTT,REALTY
52,PAGEIND,51769.93,-28.15,37.0,X-MC,6.90,81190.0,-2320.0,22352.0,-0.95,-2.78,27.53,23.98,82.0,-0.10,0.56,2.88,X40,ATH,APPARELS
53,PGHH,17907.65,-30.68,37.0,X-MC,2.88,200580.0,-240.0,68037.0,-0.26,-0.12,33.92,33.76,80.0,-0.00,1.39,5.07,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-1.97,49.0,H-LC,1.03,157420.0,-28675.0,76160.0,-0.36,-15.41,48.38,25.51,15.0,-0.38,1.09,16.01,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,62.0,X-LC,1.55,204206.0,4068.0,16827.0,-0.73,2.03,8.24,10.44,4.0,0.24,1.41,7.71,X40,NTT,FMCG
53,PGHH,17907.65,-30.68,37.0,X-MC,2.88,200580.0,-240.0,68037.0,-0.26,-0.12,33.92,33.76,80.0,-0.00,1.39,5.07,X40,ATH,FMCG
29,HAVELLS,2069.16,-6.89,42.0,X-MC,3.08,310905.0,-17086.0,123616.0,-0.81,-5.21,39.76,32.48,92.0,-0.14,2.15,5.62,X40,ATH,ELECTRICAL
86,VOLTAS,1530.00,1.02,52.0,X-MC,3.53,211860.0,20118.0,17648.0,-1.68,10.49,8.33,19.69,99.0,1.14,1.46,18.00,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COLPAL,3767.14,-5.42,42.0,X-MC,8.10,220880.0,-42485.0,155831.0,-0.35,-16.13,70.55,43.04,84.0,-0.27,1.53,2.54,XY25,ATH,FMCG
52,PAGEIND,51769.93,-28.15,37.0,X-MC,6.90,81190.0,-2320.0,22352.0,-0.95,-2.78,27.53,23.98,82.0,-0.10,0.56,2.88,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-14.90,43.0,X-MC,5.86,179236.0,-21270.0,118421.0,-0.48,-10.61,66.07,48.45,90.0,-0.18,1.24,3.36,X40N,ATH,FINANCE
3,ACC,3906.00,-37.64,51.0,X-MC,2.70,185400.0,-52551.0,205201.0,-0.47,-22.08,110.68,64.15,174.0,-0.26,1.28,4.00,XY24,BTT,CEMENT
85,VBL,671.64,-19.29,48.0,X-LC,9.51,290545.0,-25297.0,137951.0,-0.86,-8.01,47.48,35.67,5.0,-0.18,2.01,4.74,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-44.35,54.0,X-SC,38.31,55232.0,-9774.0,160703.0,3.68,-15.04,290.96,232.18,198.0,-0.06,0.38,7.73,XY24,NTT,MISC
52,PAGEIND,51769.93,-28.15,37.0,X-MC,6.90,81190.0,-2320.0,22352.0,-0.95,-2.78,27.53,23.98,82.0,-0.10,0.56,2.88,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.97,43.0,X-SC,4.67,81874.0,-63286.0,139210.0,0.13,-43.60,170.03,52.31,136.0,-0.45,0.57,9.59,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.24,44.0,X-MC,2.98,89760.0,-918.0,15825.0,0.50,-1.01,17.63,16.44,101.0,-0.06,0.62,18.96,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-37.29,38.0,X-SC,4.44,93790.0,-34880.0,80181.0,-3.10,-27.11,85.49,35.20,219.0,-0.44,0.65,7.65,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-28.04,54.0,X-LC,11.67,287142.0,-58814.0,125510.0,-0.98,-17.00,43.71,19.28,1.0,-0.47,1.98,6.15,X40,ATH,IT
40,INFY,2275.00,-18.16,56.0,X-LC,6.58,318179.0,5291.0,166408.0,-0.71,1.69,52.30,54.87,3.0,0.03,2.20,10.11,X40,BTT,IT
42,ITC,452.00,-36.66,62.0,X-LC,1.55,204206.0,4068.0,16827.0,-0.73,2.03,8.24,10.44,4.0,0.24,1.41,7.71,X40,NTT,FMCG
85,VBL,671.64,-19.29,48.0,X-LC,9.51,290545.0,-25297.0,137951.0,-0.86,-8.01,47.48,35.67,5.0,-0.18,2.01,4.74,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,51.0,H-LC,5.97,250560.0,-11059.0,130266.0,-0.34,-4.23,51.99,45.57,7.0,-0.08,1.73,6.62,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7038.89,61.0,L-SC,7.44,78310.0,-15500.0,92116.0,-0.49,-16.52,117.63,81.67,269.0,-0.17,0.54,51.65,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,27.66,81370.0,-32179.0,72232.0,0.40,-28.34,88.77,35.27,268.0,-0.45,0.56,99.15,XR,NTT,HOTELS
50,MASFIN,398.61,-20.51,45.0,H-SC,9.13,90525.0,-7455.0,29059.0,0.12,-7.61,32.10,22.05,152.0,-0.26,0.63,31.58,XR,ATH,FINANCE
87,WHIRLPOOL,2270.00,-38.39,64.0,M-SC,3.17,127779.0,12981.0,81063.0,-0.17,11.31,63.44,81.92,223.0,0.16,0.88,51.27,XR,NTT,DURABLES
47,KPIGREEN,731.05,8.64,67.0,H-SC,5.27,129629.0,4332.0,54600.0,-1.49,3.46,42.12,47.03,141.0,0.08,0.90,62.31,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-198.86,82.0,M-SC,28.60,170919.0,21699.0,122481.0,0.68,14.54,71.66,96.62,208.0,0.18,1.18,84.97,XY25,NTT,FINANCE
69,SONACOMS,806.63,-31.54,73.0,M-SC,8.28,85164.0,-15995.0,55995.0,0.58,-15.81,65.75,39.54,202.0,-0.29,0.59,20.09,AR,ATH,AUTO
5,ANGELONE,3033.00,17.35,67.0,X-SC,5.33,207640.0,16634.0,41071.0,-1.74,8.71,19.78,30.21,157.0,0.41,1.44,31.41,X40N,NTT,FINANCE
12,BANKINDIA,190.00,-16.77,74.0,H-MC,15.18,214889.0,35081.0,77339.0,0.01,19.51,35.99,62.52,88.0,0.45,1.49,60.25,XR,NTT,BANKS
47,KPIGREEN,731.05,8.64,67.0,H-SC,5.27,129629.0,4332.0,54600.0,-1.49,3.46,42.12,47.03,141.0,0.08,0.90,62.31,MH,ATH,POWER


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.32
1,25,43.39
2,50,74.12


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.89
LC    31.87
MC    24.27
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.81
X40      16.09
XY25     11.97
XR       11.29
X40N     10.47
AR        7.78
OX40N     7.48
X200      1.74
X5K       1.42
SR        1.08
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.28
X-LC    21.29
X-MC    18.54
M-SC    12.61
M-LC     5.01
X-SC     4.57
H-LC     4.56
H-MC     3.85
M-MC     1.57
L-SC     1.43
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.27,-4.45,38.53
IT,12.38,-17.44,80.43
FINANCE,12.06,-8.45,57.61
MISC,6.84,-15.86,80.05
BANKS,6.62,-4.23,59.07
ELECTRICAL,5.73,-8.94,47.56
PAINTS,5.46,-16.93,34.36
INSURANCE,3.82,-0.01,37.30
AUTO,2.76,-44.22,104.41


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3259674.0,22
XR,1316438.0,14
AR,1165425.0,9
X40,827984.0,12
X40N,645917.0,9
OX40N,566817.0,10
XY25,450604.0,8
SR,265193.0,2
X5K,112502.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3339062.0,25
M-SC,1221652.0,15
X-MC,1196887.0,14
X-LC,1057096.0,13
X-SC,519487.0,6
H-MC,324541.0,3
M-LC,284708.0,4
H-LC,278385.0,3
L-SC,254150.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1146459.0      6
           AR         835996.0      5
           XR         777399.0      7
M-SC       XY24       698457.0      6
X-MC       XY24       582037.0      4
X-LC       X40        456608.0      5
           XY24       308082.0      2
X-MC       X40        291195.0      6
X-SC       X40N       278603.0      4
H-SC       SR         265193.0      2
           OX40N      259415.0      4
X-LC       X40N       217138.0      3
H-LC       AR         206426.0      2
H-MC       XY24       178137.0      1
X-MC       XY25       173479.0      2
M-MC       XY24       165265.0      1
L-SC       XR         164348.0      2
X-SC       XY24       160703.0      1
X-MC       X40N       150176.0      2
M-SC       OX40N      148535.0      4
           XR         129176.0      2
           AR         123003.0      2
           XY25       122481.0      1
M-LC       X5K        112502.0      1
           XR         110143.0      1
L-SC       OX40N       89802.0      1
X-SC       X40         80181.0      1
H-MC       XR          77339.0      1
X-LC       XY25        75268.0      3
H-LC       X200        71959.0      1
H-MC       OX40N       69065.0      1
L-MC       XR          58033.0      1
H-SC       MH          54600.0      1
M-LC       XY25        41529.0      1
L-LC       XY25        37847.0      1
M-LC       XY24        20534.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
